In [1]:
import os
from pathlib import Path
from PIL import Image
import cv2
import numpy as np
import json
from shapely.geometry import Polygon
import image_slicer

# DeepLab code:
# taken from https://gluon-cv.mxnet.io/build/examples_segmentation/demo_deeplab.html
# dataset description https://groups.csail.mit.edu/vision/datasets/ADE20K/, https://github.com/dmlc/gluon-cv/blob/master/gluoncv/data/ade20k/segmentation.py
# deeplab code https://github.com/dmlc/gluon-cv/blob/master/gluoncv/model_zoo/deeplabv3.py

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
from gluoncv.data.transforms.presets.segmentation import test_transform
# using cpu
ctx = mx.cpu(0)

# OCR code
import pytesseract
from pytesseract import Output


# If you don't have tesseract executable in your PATH, include the following:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' # Path('C:/Program Files\ Tesseract-OCR\ tesseract').as_posix()


# Scene recognition imports
from Keras_VGG16_places365.vgg16_places_365 import VGG16_Places365
from cv2 import resize


import evaluation_utils as eval_u
import deeplab_utils as dl_u

Using TensorFlow backend.


In [2]:
image_list_deeplab, list_image_names_deeplab = eval_u.prepare_dataset('../small_10_test2017/', 'deeplab')

In [ ]:
model_deeplab = eval_u.load_model('deeplab')
output_pred_deeplab = eval_u.get_predictions(image_list_deeplab, 'deeplab', model_deeplab)

Currently treating image number  1
Currently treating image number  2
Currently treating image number  3
Currently treating image number  4


In [ ]:
print(output_pred_deeplab[0].shape)

In [ ]:
#outp = mx.nd.squeeze(mx.nd.argmax(output_pred_deeplab[0], 1)).asnumpy()


In [7]:
output = dl_u.deeplab_pred_to_output(outp)

prepare masks
Conpute unque masks.
mask computed
[  0.   2.  12.  14.  21.  26.  34. 126.]
prepare list polygons


In [10]:
def simple_rule_based_obfuscation(output_deeplab):
    # Later we can save other things like the "sky" to decide to hide it
    true_output = []
    for poly in output_deeplab:
        if (poly[1] == "person, individual, someone, somebody, mortal, soul") or (poly[1] == "bookcase") or (poly[1] == "computer, computing machine, computing device, data processor, ") or (poly[1] == "trade name, brand name, brand, marque") :
            true_output.append(poly)
    return true_output

In [ ]:
print(type(output_pred_deeplab[0]))

In [8]:
print(output)

[([<shapely.geometry.polygon.Polygon object at 0x0000020281C99D88>, <shapely.geometry.polygon.Polygon object at 0x00000202F84AA048>, <shapely.geometry.polygon.Polygon object at 0x0000020281C99EC8>, <shapely.geometry.polygon.Polygon object at 0x0000020281C99F08>, <shapely.geometry.polygon.Polygon object at 0x0000020281C99F48>, <shapely.geometry.polygon.Polygon object at 0x0000020281C99E88>, <shapely.geometry.polygon.Polygon object at 0x0000020281C99FC8>, <shapely.geometry.polygon.Polygon object at 0x0000020281C99E48>, <shapely.geometry.polygon.Polygon object at 0x0000020281C99DC8>, <shapely.geometry.polygon.Polygon object at 0x0000020281C99F88>, <shapely.geometry.polygon.Polygon object at 0x0000020282CCD0C8>, <shapely.geometry.polygon.Polygon object at 0x0000020282CCD148>, <shapely.geometry.polygon.Polygon object at 0x0000020282CCD188>, <shapely.geometry.polygon.Polygon object at 0x0000020282CCD1C8>, <shapely.geometry.polygon.Polygon object at 0x0000020282CCD208>, <shapely.geometry.poly

In [11]:
true_output = simple_rule_based_obfuscation(output)

In [12]:
print(true_output)

[([<shapely.geometry.polygon.Polygon object at 0x0000020282CCDD08>, <shapely.geometry.polygon.Polygon object at 0x0000020282CCD108>, <shapely.geometry.polygon.Polygon object at 0x0000020282CCD088>, <shapely.geometry.polygon.Polygon object at 0x0000020282CCD048>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD06C8>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD0688>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD0788>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD07C8>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD0808>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD0708>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD0748>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD0888>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD08C8>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD0908>, <shapely.geometry.polygon.Polygon object at 0x0000020282CD0948>, <shapely.geometry.poly

In [13]:
with open(Path('../test2017.json'), 'r') as f:
    ground_truth = json.load(f)
ground_truth = ground_truth['annotations']

In [ ]:
eval_u.evaluate(ground_truth, true_output, evaluation_type, parameter_interval)